In [20]:
import numpy as np
import pandas as pd
import re,string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import joblib

In [21]:
df=pd.read_csv("Restaurant_Reviews.tsv",sep='\t')

In [22]:
df.dtypes

Review    object
Liked      int64
dtype: object

In [23]:
X=df['Review']
y=df['Liked']
X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=42)

In [24]:
def preprocessor(data):
    sw= stopwords.words('english')
    sw.remove('not')
    sw.remove("don't")
    sw.remove("shouldn't")
    sw.remove("wouldn't")
    lemma=WordNetLemmatizer()
    ps = PorterStemmer()
    
    re.sub(r"http\S+|www.*", "", data)#remove url links
    data = word_tokenize(data.lower())#lowercase and tokenize the words
    data = [word for word in set(data) if word.isalpha() and word not in string.punctuation and word!='\n' and len(word) > 2 and word not in sw]

    return " ".join(data)

In [25]:
cv=CountVectorizer(lowercase=True,preprocessor=preprocessor)
pipeline1=Pipeline([('Vectorizer',cv),('model',SVC())])
pipeline1.fit(X_train,y_train)

Pipeline(steps=[('Vectorizer',
                 CountVectorizer(preprocessor=<function preprocessor at 0x0000013E53073A30>)),
                ('model', SVC())])

In [35]:
joblib.dump(cv, "cv.pkl")

['cv.pkl']

In [36]:
y_pred = pipeline1.predict(X_test)

In [37]:
('Accuracy: {} %'.format(100 * accuracy_score(y_test, y_pred)))

'Accuracy: 80.80000000000001 %'

In [38]:
pipeline2=Pipeline([('Vectorizer',cv),('model',MultinomialNB())])
pipeline2.fit(X_train,y_train)

Pipeline(steps=[('Vectorizer',
                 CountVectorizer(preprocessor=<function preprocessor at 0x0000013E53073A30>)),
                ('model', MultinomialNB())])

In [39]:
y_pred2 = pipeline2.predict(X_test)
('Accuracy: {} %'.format(100 * accuracy_score(y_test, y_pred2)))

'Accuracy: 80.4 %'

In [31]:
joblib.dump(pipeline2,"finalmodel.sav")

['finalmodel.sav']